In [ ]:
# This cell is mandatory in all Dymos documentation notebooks.
missing_packages = []
try:
    import openmdao.api as om  # noqa: F401
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !python -m pip install openmdao[notebooks]
    else:
        missing_packages.append('openmdao')
try:
    import dymos as dm  # noqa: F401
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !python -m pip install dymos
    else:
        missing_packages.append('dymos')
try:
    import pyoptsparse  # noqa: F401
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !pip install -q condacolab
        import condacolab
        condacolab.install_miniconda()
        !conda install -c conda-forge pyoptsparse
    else:
        missing_packages.append('pyoptsparse')
if missing_packages:
    raise EnvironmentError('This notebook requires the following packages '
                           'please install them and restart this notebook\'s runtime: {",".join(missing_packages)}')


(examples:balanced_field_solved)=

# Aircraft Balanced Field Length Calculation without an Optimizer

```{admonition} Things you'll learn through this example
- Solving complex trajectories without an optimizer.
```

This example is similar  to that of the previous balanced field example with a few changes.

First, there are no control variables. Instead, things like pitch rates of the vehicle are assumed.
Secondly, we use PicardShooting to propagate the equations of motion.

The duration of phases is govered by associated thim with a balanced that targets some final condition with the `add_boundary_balance` method.
The phases are linked directly by connection rather than by optimizer constraint.

So, given a guess at the decision speed, `V1`, dymos propagates both the "reject takeoff" branch and the "proceed with takeoff under one engine branch".

Finally, we make `V1` the output of a BalanceComp in the trajectory such that a NewtonSolver can iterate on its value until the range at the end of the two branches is equal. Recall that a Trajectory is just an OpenMDAO Group with some dymos-specific components within it. We can add a BalancedComponent to the end of it and assign solvers without interfering with the functionality of dymos.

This problem is set up to give you an example of how to might solve some relatively complicated sequencing using dymos without an optimizer.

## The ODE System

In this example, we're going to use the same ODE as in the other balanced field example, but we're going to build it with `jax` so that we have derivatives computing using automatic differentiation.

### BalancedFieldODEComp

In [ ]:
import jax.numpy as jnp


class BalancedFieldJaxODEComp(om.JaxExplicitComponent):
    """
    The ODE System for an aircraft takeoff climb.

    Computes the rates for states v (true airspeed) gam (flight path angle) r (range) and h (altitude).

    References
    ----------
    .. [1] Raymer, Daniel. Aircraft design: a conceptual approach. American Institute of
    Aeronautics and Astronautics, Inc., 2012.
    """
    def initialize(self):
        self.options.declare('num_nodes', types=int)
        self.options.declare('g', types=(float, int), default=9.80665, desc='gravitational acceleration (m/s**2)')
        self.options.declare('mode', values=('runway', 'climb'), desc='mode of operation (ground roll or flight)')
        self.options.declare('attitude_input', values=('pitch', 'alpha'), default='alpha', desc='attitude input type')
        self.options.declare('control', values=('attitude', 'gam_rate'), default='gam_rate',
                             desc='Whether pitch or alpha serves as control when climbing.')

    def get_self_statics(self):
        return (self.options['num_nodes'], self.options['g'], self.options['mode'],
                self.options['attitude_input'], self.options['control'])

    def setup(self):
        nn = self.options['num_nodes']

        # Scalar (constant) inputs
        self.add_input('rho', val=1.225, desc='atmospheric density at runway', units='kg/m**3')
        self.add_input('S', val=124.7, desc='aerodynamic reference area', units='m**2')
        self.add_input('CD0', val=0.03, desc='zero-lift drag coefficient', units=None)
        self.add_input('CL0', val=0.5, desc='zero-alpha lift coefficient', units=None)
        self.add_input('CL_max', val=2.0, desc='maximum lift coefficient for linear fit', units=None)
        self.add_input('alpha_max', val=jnp.radians(10), desc='angle of attack at CL_max', units='rad')
        self.add_input('h_w', val=1.0, desc='height of the wing above the CG', units='m')
        self.add_input('AR', val=9.45, desc='wing aspect ratio', units=None)
        self.add_input('e', val=0.801, desc='Oswald span efficiency factor', units=None)
        self.add_input('span', val=35.7, desc='Wingspan', units='m')
        self.add_input('T', val=1.0, desc='thrust', units='N')
        self.add_input('mu_r', val=0.05, desc='runway friction coefficient', units=None)

        # Dynamic inputs (can assume a different value at every node)
        self.add_input('m', shape=(nn,), desc='aircraft mass', units='kg')
        self.add_input('v', shape=(nn,), desc='aircraft true airspeed', units='m/s')
        self.add_input('h', shape=(nn,), desc='altitude', units='m')
        self.add_input('gam', shape=(nn,), val=0.0, desc='flight path angle', units='rad')
        self.add_input('pitch', shape=(nn,), desc='vehicle +x angle above horizon', units='rad')

        self.add_input('gam_rate', shape=(nn,), val=0.0, desc='controlled rate of change of flight path angle', units='rad/s')

        # Outputs
        self.add_output('alpha', shape=(nn,), desc='angle of attack', units='rad')
        self.add_output('CL', shape=(nn,), desc='lift coefficient', units=None)
        self.add_output('q', shape=(nn,), desc='dynamic pressure', units='Pa')
        self.add_output('L', shape=(nn,), desc='lift force', units='N')
        self.add_output('D', shape=(nn,), desc='drag force', units='N')
        self.add_output('K', val=jnp.ones(nn), desc='drag-due-to-lift factor', units=None)
        self.add_output('F_r', shape=(nn,), desc='runway normal force', units='N')
        self.add_output('v_dot', shape=(nn,), desc='rate of change of speed', units='m/s**2',
                        tags=['dymos.state_rate_source:v'])
        self.add_output('r_dot', shape=(nn,), desc='rate of change of range', units='m/s',
                        tags=['dymos.state_rate_source:r'])
        self.add_output('W', shape=(nn,), desc='aircraft weight', units='N')
        self.add_output('v_stall', shape=(nn,), desc='stall speed', units='m/s')
        self.add_output('v_over_v_stall', shape=(nn,), desc='stall speed ratio', units=None)
        self.add_output('climb_gradient', shape=(nn,),
                        desc='altitude rate divided by range rate',
                        units='unitless')
        self.add_output('gam_dot', shape=(nn,), desc='rate of change of flight path angle', units='rad/s')
        self.add_output('h_dot', shape=(nn,), desc='rate of change of altitude', units='m/s')

        self.declare_coloring(wrt='*', method='jax', show_summary=False)

    def compute_primal(self, rho, S, CD0, CL0, CL_max, alpha_max, h_w, AR, e, span, T, mu_r, m, v, h, gam, pitch, gam_rate):
        g = self.options['g']

        alpha = alpha = pitch - gam

        W = m * g
        v_stall = jnp.sqrt(2 * W / rho / S / CL_max)
        v_over_v_stall = v / v_stall

        CL = CL0 + (alpha / alpha_max) * (CL_max - CL0)
        K_nom = 1.0 / (jnp.pi * AR * e)
        b = span / 2.0

        # Note the use of clip here.  If altitude drops below zero while the solver is iterating,
        # the non-clipped equation will result in NaN and ruin the analysis.
        # Since we're using a gradient-free nonlinear block GS to converge thedo we n
        fact = (jnp.clip(jnp.real(h + h_w), 0.0, 1000.0) / b) ** 1.5
        K = K_nom * 33 * fact / (1.0 + 33 * fact)

        CD = (CD0 + K * CL ** 2)

        # CL, CD = ground_effect(h, pitch, CD0, CL, CD, AR, h_w, span)

        q = 0.5 * rho * v ** 2
        L = q * S * CL
        D = q * S * CD

        # Compute the downward force on the landing gear
        calpha = jnp.cos(alpha)
        salpha = jnp.sin(alpha)

        # Runway normal force
        if self.options['mode'] == 'runway':
            F_r = m * g - L * calpha - T * salpha
        else:
            F_r = 0.0

        #  Compute the dynamics
        cgam = jnp.cos(gam)
        sgam = jnp.sin(gam)
        v_dot = (T * calpha - D - F_r * mu_r) / m - g * sgam
        h_dot = v * sgam
        r_dot = v * cgam
        climb_gradient = sgam

        if self.options['control'] == 'gam_rate':
            gam_dot = gam_rate
        else:
            gam_dot = (T * salpha + L) / (m * v) - (g / v) * cgam

        return alpha, CL, q, L, D, K, F_r, v_dot, r_dot, W, v_stall, v_over_v_stall, climb_gradient, gam_dot, h_dot


## Building and running the problem

In the following code we define and solve the optimal control problem.
First we define the problem and specify the optimizer settings.

### Phase: Brake Release to Engine Failure

In this phase, we propagate in runway mode with a fixed pitch of zero degrees.
We add a calculation of our "velocity-to-go" as the difference between our current airspeed and the airspeed at which an engine fails.
This in turn allows us to specify a residual for `t_duration` for the phase, such that this `velocity-to-go` is zero.

In [ ]:
import time
import openmdao.api as om
import dymos as dm

NODES_PER_SEG = 9

p = om.Problem()

# Brake release to v_ef - both engines operable
tx = dm.PicardShooting(num_segments=1, nodes_per_seg=NODES_PER_SEG)
br_to_vef = dm.Phase(ode_class=BalancedFieldJaxODEComp, transcription=tx,
                        ode_init_kwargs={'mode': 'runway',
                                        'attitude_input': 'pitch',
                                        'control': 'gam_rate'})
br_to_vef.set_time_options(fix_initial=True, fix_duration=True, duration_bounds=(1, 1000))
br_to_vef.add_state('r', fix_initial=True, lower=0)
br_to_vef.add_state('v', fix_initial=True, lower=0)
br_to_vef.add_parameter('pitch', val=0.0, opt=False, units='deg')
br_to_vef.add_parameter('v_ef', val=100.0, opt=False, units='kn')
br_to_vef.add_calc_expr('v_to_go = v - v_ef',
                        v={'units': 'kn'},
                        v_ef={'units': 'kn'},
                        v_to_go={'units': 'kn'})
br_to_vef.add_boundary_balance(param='t_duration', name='v_to_go', tgt_val=0.0, loc='final')
br_to_vef.add_timeseries_output('*')

### Phase: Engine Failure to Decision Speed (V1)

The next phase uses a similar setup, propagating until the decision speed (V1) is reached.

In [ ]:
# Engine failure to v1 - decision reaction time
tx = dm.PicardShooting(num_segments=1, nodes_per_seg=NODES_PER_SEG)
vef_to_v1 = dm.Phase(ode_class=BalancedFieldJaxODEComp, transcription=tx,
                        ode_init_kwargs={'mode': 'runway',
                                        'attitude_input': 'pitch',
                                        'control': 'gam_rate'})
vef_to_v1.set_time_options(fix_initial=True, fix_duration=True)
vef_to_v1.add_state('r', fix_initial=True, lower=0)
vef_to_v1.add_state('v', fix_initial=True, lower=0)
vef_to_v1.add_parameter('pitch', val=0.0, opt=False, units='deg')
vef_to_v1.add_parameter('v1', val=150.0, opt=False, units='kn')
vef_to_v1.add_calc_expr('v_to_go = v - v1',
                        v={'units': 'kn'},
                        v1={'units': 'kn'},
                        v_to_go={'units': 'kn'})
vef_to_v1.add_boundary_balance(param='t_duration', name='v_to_go', tgt_val=0.0, loc='final')
vef_to_v1.add_timeseries_output('*')

### Phase: Reject Takeoff

The next phase is a branch of the trajectory that starts at V1 and propagates under braking until the airspeed is zero.

In [ ]:
# Rejected takeoff at V1 - no engines operable - decelerate to stop
tx = dm.PicardShooting(num_segments=1, nodes_per_seg=NODES_PER_SEG)
rto = dm.Phase(ode_class=BalancedFieldJaxODEComp, transcription=tx,
                ode_init_kwargs={'mode': 'runway',
                                'attitude_input': 'pitch',
                                'control': 'gam_rate'})
rto.set_time_options(fix_initial=True, fix_duration=True)
rto.add_state('r', fix_initial=False, lower=0)
rto.add_state('v', fix_initial=False, lower=0)
rto.add_parameter('pitch', val=0.0, opt=False, units='deg')
rto.add_boundary_balance(param='t_duration', name='v', tgt_val=0.0, loc='final')
rto.add_timeseries_output('*')

### Phase: V1 to Rotation

The V1 to rotation phase propagates until the ratio of our airspeed over the stall speed is 1.11.
At this point, the aicraft will begin pitching up for takeoff.

In [ ]:
# V1 to Vr - Single Engine Operable
tx = dm.PicardShooting(num_segments=1, nodes_per_seg=NODES_PER_SEG)
v1_to_vr = dm.Phase(ode_class=BalancedFieldJaxODEComp, transcription=tx,
                        ode_init_kwargs={'mode': 'runway',
                                        'attitude_input': 'pitch',
                                        'control': 'gam_rate'})
v1_to_vr.set_time_options(fix_initial=True, fix_duration=True)
v1_to_vr.add_state('r', fix_initial=False, lower=0)
v1_to_vr.add_state('v', fix_initial=False, lower=0)
v1_to_vr.add_parameter('pitch', val=0.0, opt=False, units='deg')
v1_to_vr.add_boundary_balance(param='t_duration', name='v_over_v_stall', tgt_val=1.11)
v1_to_vr.add_timeseries_output('*')

### Phase: Rotation until takeoff

In the rotation phase, the aicraft pitches at a constant rate.  With sufficient airspeed, the aircraft will produce enough lift to cancel out its weight and it will "unstick" from the runway.

In [ ]:
# Rotation to liftoff - single engine operable
tx = dm.PicardShooting(num_segments=1, nodes_per_seg=NODES_PER_SEG)
rotate = dm.Phase(ode_class=BalancedFieldJaxODEComp, transcription=tx,
                ode_init_kwargs={'mode': 'runway',
                                'attitude_input': 'pitch',
                                'control': 'gam_rate'})
rotate.set_time_options(fix_initial=True, fix_duration=True)
rotate.add_state('r', fix_initial=False, lower=0)
rotate.add_state('v', fix_initial=False, lower=0)
rotate.add_state('pitch', rate_source='pitch_rate', fix_initial=True, lower=0, upper=15, units='deg')
rotate.add_parameter('pitch_rate', opt=False, units='deg/s')
rotate.add_boundary_balance(param='t_duration', name='F_r', tgt_val=0.0, )
rotate.add_timeseries_output('*')
rotate.add_timeseries_output('alpha', units='deg')

### Phase: Liftoff to Climb Gradient

In this phase, we continue to pitch up until the aircraft achieves some desired climb gradient.
The climb gradient is just the sine of the flight path angle.

In [ ]:
# Liftoff and rotate until climb gradient is achieved
tx = dm.PicardShooting(num_segments=1, nodes_per_seg=NODES_PER_SEG)
liftoff_to_climb_gradient = dm.Phase(ode_class=BalancedFieldJaxODEComp, transcription=tx,
                                        ode_init_kwargs={'mode': 'runway',
                                                        'attitude_input': 'pitch',
                                                        'control': 'attitude'})
liftoff_to_climb_gradient.set_time_options(fix_initial=True, fix_duration=True)
liftoff_to_climb_gradient.set_state_options('r', fix_initial=True, lower=0)
liftoff_to_climb_gradient.set_state_options('h', fix_initial=True, rate_source='h_dot', lower=0)
liftoff_to_climb_gradient.set_state_options('v', fix_initial=True, lower=0)
liftoff_to_climb_gradient.set_state_options('gam', fix_initial=True, rate_source='gam_dot', lower=0)
liftoff_to_climb_gradient.set_state_options('pitch', fix_initial=True, rate_source='pitch_rate', opt=False, units='deg')
liftoff_to_climb_gradient.add_parameter('pitch_rate', opt=False, units='deg/s')
liftoff_to_climb_gradient.add_parameter('mu_r', opt=False, val=0.0, units=None)
liftoff_to_climb_gradient.add_boundary_balance(param='t_duration', name='climb_gradient', tgt_val=0.05, loc='final', lower=0, upper=10)
liftoff_to_climb_gradient.add_timeseries_output('alpha', units='deg')
liftoff_to_climb_gradient.add_timeseries_output('h', units='ft')
liftoff_to_climb_gradient.add_timeseries_output('*')

### Phase: Climb to Obstacle Clearance

Finally, having achieved the desired climb gradient, we maintain that climb gradient until we are 35 feet above the ground.

If we choose our decision speed correctly, the range at which we clear this 35 foot altitude requirement will be equal to the range at wheels-stop in the rejected takeoff phase. This is controlled by our phase linkage definitions a few sections down.

In [ ]:
# Sixth Phase: Assume constant flight path angle until 35ft altitude is reached.
tx = dm.PicardShooting(num_segments=1, nodes_per_seg=NODES_PER_SEG)
climb_to_obstacle_clearance = dm.Phase(ode_class=BalancedFieldJaxODEComp, transcription=tx,
                                        ode_init_kwargs={'mode': 'runway',
                                                        'attitude_input': 'pitch',
                                                        'control': 'gam_rate'})
climb_to_obstacle_clearance.set_time_options(fix_initial=True, fix_duration=True)
climb_to_obstacle_clearance.set_state_options('r', fix_initial=True, lower=0)
climb_to_obstacle_clearance.set_state_options('h', fix_initial=True, rate_source='h_dot', lower=0)
climb_to_obstacle_clearance.set_state_options('v', fix_initial=True, lower=0)
climb_to_obstacle_clearance.set_state_options('gam', fix_initial=True, rate_source='gam_dot', lower=0)
climb_to_obstacle_clearance.set_state_options('pitch', fix_initial=True, rate_source='pitch_rate', opt=False, units='deg')
climb_to_obstacle_clearance.add_parameter('pitch_rate', opt=False, units='deg/s')
climb_to_obstacle_clearance.add_parameter('gam_rate', opt=False, units='deg/s')
climb_to_obstacle_clearance.add_parameter('mu_r', opt=False, val=0.0, units=None)
climb_to_obstacle_clearance.add_boundary_balance(param='t_duration', name='h', tgt_val=35, eq_units='ft', loc='final', lower=0.01, upper=100)
climb_to_obstacle_clearance.add_timeseries_output('alpha', units='deg')
climb_to_obstacle_clearance.add_timeseries_output('h', units='ft')
climb_to_obstacle_clearance.add_timeseries_output('*')

Create our trajectory, add phases to it, and then add some trajectory-level parameters that will be passed to various phases.

In [ ]:
# Instantiate the trajectory and add phases
traj = dm.Trajectory(parallel_phases=False)
p.model.add_subsystem('traj', traj)
traj.add_phase('br_to_vef', br_to_vef)
traj.add_phase('vef_to_v1', vef_to_v1)
traj.add_phase('rto', rto)
traj.add_phase('v1_to_vr', v1_to_vr)
traj.add_phase('rotate', rotate)
traj.add_phase('liftoff_to_climb_gradient', liftoff_to_climb_gradient)
traj.add_phase('climb_to_obstacle_clearance', climb_to_obstacle_clearance)

# Add parameters common to multiple phases to the trajectory
traj.add_parameter('m', val=174200., opt=False, units='lbm',
                desc='aircraft mass',
                targets={phase: ['m'] for phase in
                                ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate',
                                'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})

traj.add_parameter('T_nominal', val=27000 * 2, opt=False, units='lbf', static_target=True,
                desc='nominal aircraft thrust',
                targets={'br_to_vef': ['T']})

traj.add_parameter('T_engine_out', val=27000, opt=False, units='lbf', static_target=True,
                desc='thrust under a single engine',
                targets={phase: ['T'] for phase in ['vef_to_v1', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})

traj.add_parameter('T_shutdown', val=0.0, opt=False, units='lbf', static_target=True,
                desc='thrust when engines are shut down for rejected takeoff',
                targets={'rto': ['T']})

traj.add_parameter('mu_r_nominal', val=0.03, opt=False, units=None, static_target=True,
                desc='nominal runway friction coefficient',
                targets={phase: ['mu_r'] for phase in ['br_to_vef', 'vef_to_v1', 'v1_to_vr', 'rotate']})

traj.add_parameter('mu_r_braking', val=0.3, opt=False, units=None, static_target=True,
                desc='runway friction coefficient under braking',
                targets={'rto': ['mu_r']})

traj.add_parameter('h_runway', val=0., opt=False, units='ft',
                desc='runway altitude',
                targets={phase: ['h'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate']})

traj.add_parameter('rho', val=1.225, opt=False, units='kg/m**3', static_target=True,
                desc='atmospheric density',
                targets={phase: ['rho'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})

traj.add_parameter('S', val=124.7, opt=False, units='m**2', static_target=True,
                desc='aerodynamic reference area',
                targets={f'{phase}': ['S'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('CD0', val=0.03, opt=False, units=None, static_target=True,
                desc='zero-lift drag coefficient',
                targets={f'{phase}': ['CD0'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('AR', val=9.45, opt=False, units=None, static_target=True,
                desc='wing aspect ratio',
                targets={f'{phase}': ['AR'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('e', val=801, opt=False, units=None, static_target=True,
                desc='Oswald span efficiency factor',
                targets={f'{phase}': ['e'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('span', val=35.7, opt=False, units='m', static_target=True,
                desc='wingspan',
                targets={f'{phase}': ['span'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('h_w', val=1.0, opt=False, units='m', static_target=True,
                desc='height of wing above CG',
                targets={f'{phase}': ['h_w'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('CL0', val=0.5, opt=False, units=None, static_target=True,
                desc='zero-alpha lift coefficient',
                targets={f'{phase}': ['CL0'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('CL_max', val=2.0, opt=False, units=None, static_target=True,
                desc='maximum lift coefficient for linear fit',
                targets={f'{phase}': ['CL_max'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',

traj.add_parameter('alpha_max', val=10.0, opt=False, units='deg', static_target=True,
                desc='angle of attack at maximum lift',
                targets={f'{phase}': ['alpha_max'] for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']})#, 'v1_to_vr',


## Linking Phases

Now we link our phases. It's important to note that we're linking by connection (rather than constraint) because we don't intend to use an optimizer to compute this trajectory.  There are no design variables...all controls are based on assumed pitch or flight path angle rates.

In [ ]:
# Linkages are done via connection to avoid the need for an optimizer to resolve the linkage constraints.
traj.link_phases(['br_to_vef', 'vef_to_v1'], vars=['time', 'r', 'v'], connected=True)
traj.link_phases(['vef_to_v1', 'rto'], vars=['time', 'r', 'v'], connected=True)
traj.link_phases(['vef_to_v1', 'v1_to_vr'], vars=['time', 'r', 'v'], connected=True)
traj.link_phases(['v1_to_vr', 'rotate'], vars=['time', 'r', 'v'], connected=True)
traj.link_phases(['rotate', 'liftoff_to_climb_gradient'], vars=['time', 'r', 'v', 'pitch'], connected=True)
traj.link_phases(['liftoff_to_climb_gradient', 'climb_to_obstacle_clearance'],
                 vars=['time', 'h', 'gam', 'r', 'v', 'pitch'],
                 connected=True)

## Determining Decision Speed and Engine Failure Speed

Our V1 speed is determined by varying it such that the range after a rejected takeoff comes to a stop is the same as the range after a single-engine takeoff clears the 35 foot obstacle requirement.

This can be accomplished by using a balance component.
We _could_ put this balanced component after the trajectory and feed V1 back to the second phase.
Alternatively, since the Trajectory object is just an OpenMDAO group with some dymos-specific components, we can add the balance comp there.
Because the internals of that Trajectory don't exist until after setup, we use the auto-ordering feature of OpenMDAO to order the trajectory components correctly.  OpenMDAO will determine the necessary ordering such that none of the groups in order to properly handle the feedback of V1.

We also add a balance to compute the velocity of engine failure, `V_ef`.  This is set such that the duration of `vef_to_v1` is equal to the parameter `t_react`, the reaction time of the pilot in deciding to either abort the takeoff or perform a single-engine takeoff. Later, we will set the value of `t_react` to one second.


In [ ]:

# We need a balance comp to satisfy residuals that cannot be satisfied by connection.
# - the final range of `rto` and `climb_to_obstacle_clearance` match
# - the velocity at engine failure is such that the time from engine failure to v1 is the reaction time.
#
# Since the trajectory components are added during the setup/configure process, this
# balance will show up as the first item in the trajectory.
traj_balance_comp = om.BalanceComp()
traj.add_subsystem('traj_balance_comp', traj_balance_comp)
traj.options['auto_order'] = True

# Vary v1 such that the final range after RTO is the same as the final range after achieving 35ft altitude.
traj_balance_comp.add_balance('v1', units='kn', eq_units='ft', val=130, lower=50, upper=180, lhs_name='r_obstacle', rhs_name='r_rto')
traj.connect('rto.final_states:r', 'traj_balance_comp.r_rto')
traj.connect('climb_to_obstacle_clearance.final_states:r', 'traj_balance_comp.r_obstacle')
traj.connect('v1', 'vef_to_v1.parameters:v1')
traj.promotes('traj_balance_comp', outputs=['v1'])

# Vary v_ef such that the time from engine failure to v1 is the reaction time.
traj_balance_comp.add_balance('v_ef', units='kn', eq_units='s', val=80, lower=20, upper=180, lhs_name='ef_to_v1_duration', rhs_name='t_react')
traj.connect('vef_to_v1.t_duration_val', 'traj_balance_comp.ef_to_v1_duration')
traj.connect('v_ef', 'br_to_vef.parameters:v_ef')
traj.promotes('traj_balance_comp', inputs=['t_react'], outputs=['v_ef'])

traj.nonlinear_solver = om.NewtonSolver(solve_subsystems=True, iprint=2, maxiter=100)
traj.nonlinear_solver.linesearch = om.ArmijoGoldsteinLS()
traj.linear_solver = om.DirectSolver()

#
# Setup the problem and set the initial guess
#
p.setup()

traj.set_val('t_react', 1.0, units='s')
traj.set_val('v_ef', 130.0, units='kn')

br_to_vef.set_time_val(initial=0.0, duration=35.0)
br_to_vef.set_state_val('r', [0, 2000.0])
br_to_vef.set_state_val('v', [0.0, 80.0])
br_to_vef.set_parameter_val('pitch', 0.0, units='deg')

vef_to_v1.set_time_val(initial=0.0, duration=35.0)
vef_to_v1.set_state_val('r', [2000, 2500.0])
vef_to_v1.set_state_val('v', [80.0, 100.0])
vef_to_v1.set_parameter_val('pitch', 0.0, units='deg')
vef_to_v1.set_parameter_val('v1', 140.0, units='kn')

rto.set_time_val(initial=30.0, duration=35.0)
rto.set_state_val('r', [2500, 5000.0])
rto.set_state_val('v', [100, 0.0], units='kn')
rto.set_parameter_val('pitch', 0.0, units='deg')

rotate.set_time_val(initial=30.0, duration=5.0)
rotate.set_state_val('r', [1750, 1800.0])
rotate.set_state_val('v', [80, 85.0])
rotate.set_state_val('pitch', [0.0, 10], units='deg')
rotate.set_parameter_val('pitch_rate', val=2.0, units='deg/s')

liftoff_to_climb_gradient.set_time_val(initial=35.0, duration=4.1)
liftoff_to_climb_gradient.set_state_val('r', [1800, 2000.0], units='ft')
liftoff_to_climb_gradient.set_state_val('v', [160, 170.0], units='kn')
liftoff_to_climb_gradient.set_state_val('h', [0.0, 35.0], units='ft')
liftoff_to_climb_gradient.set_state_val('gam', [0.0, 5.0], units='deg')
liftoff_to_climb_gradient.set_state_val('pitch', [5.0, 15.0], units='deg')
liftoff_to_climb_gradient.set_parameter_val('pitch_rate', 2.0, units='deg/s')
liftoff_to_climb_gradient.set_parameter_val('mu_r', 0.0, units=None)

climb_to_obstacle_clearance.set_time_val(initial=40.0, duration=2)
climb_to_obstacle_clearance.set_state_val('r', [2000, 5000.0], units='ft')
climb_to_obstacle_clearance.set_state_val('v', [160, 170.0], units='kn')
climb_to_obstacle_clearance.set_state_val('h', [25.0, 35.0], units='ft')
climb_to_obstacle_clearance.set_state_val('gam', [0.0, 5.0], units='deg')
climb_to_obstacle_clearance.set_state_val('pitch', [5.0, 15.0], units='deg')
climb_to_obstacle_clearance.set_parameter_val('pitch_rate', 2.0, units='deg/s')
climb_to_obstacle_clearance.set_parameter_val('gam_rate', 0.0, units='deg/s')
climb_to_obstacle_clearance.set_parameter_val('mu_r', 0.0, units=None)

start = time.perf_counter()
dm.run_problem(p, run_driver=False, simulate=True, make_plots=True)
end = time.perf_counter()

elapsed_time = end - start
print(f'Elapsed time: {elapsed_time:.6f} seconds')

t_react = p.get_val('traj.t_react', units='s')[0]
v_ef = p.get_val('traj.v_ef', units='kn')[0]
v1 = p.get_val('traj.v1', units='kn')[0]
vr = p.get_val('traj.rotate.initial_states:v', units='kn')[0]
v2 = p.get_val('traj.climb_to_obstacle_clearance.final_states:v', units='kn')[0, 0]
v_over_v_stall = p.get_val('traj.climb_to_obstacle_clearance.timeseries.v_over_v_stall')[-1, 0]
runway_length = p.get_val('traj.rto.final_states:r', units='m')[0, 0]
obstacle_dist = p.get_val('traj.climb_to_obstacle_clearance.final_states:r', units='m')[0, 0]

print(f"{'Balanced Field Length':<21}: {runway_length:6.1f} ft")
print(f"{'V_ef':<21}: {v_ef:6.1f} kts")
print(f"{'V_1':<21}: {v1:6.1f} kts")
print(f"{'V_R':<21}: {vr:6.1f} kts")
print(f"{'V_2':<21}: {v2:6.1f} kts ({v_over_v_stall:4.3f} * v_stall)")
print(f"{'Assumed Reaction Time':<21}: {t_react:6.1f} s")

The figure below shows the resulting simulated trajectory and confirms that range at the end of phase `rto` is equal to `range` at the end of climb.
This is the shortest possible field that accommodates both a rejected takeoff and a takeoff after a single engine failure at V1.

The following input cell may be expanded to see how we plotted the data.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

sim_prob = traj.sim_prob
sim_case = om.CaseReader(sim_prob.get_outputs_dir() / 'dymos_simulation.db').get_case('final')

fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'top': 0.92}, figsize=(12,6))

for phase in ['br_to_vef', 'vef_to_v1', 'rto', 'v1_to_vr', 'rotate', 'liftoff_to_climb_gradient', 'climb_to_obstacle_clearance']:
    r = sim_case.get_val(f'traj.{phase}.timeseries.r', units='ft')
    v = sim_case.get_val(f'traj.{phase}.timeseries.v', units='kn')
    t = sim_case.get_val(f'traj.{phase}.timeseries.time', units='s')
    axes[0].plot(t, r, '-', label=phase)
    axes[1].plot(t, v, '-', label=phase)
fig.suptitle('Balanced Field Length')
axes[1].set_xlabel('time (s)')
axes[0].set_ylabel('range (ft)')
axes[1].set_ylabel('airspeed (kts)')
axes[0].grid(True)
axes[1].grid(True)

tv1 = sim_case.get_val('traj.vef_to_v1.timeseries.time', units='s')[-1, 0]
v1 = sim_case.get_val('traj.vef_to_v1.timeseries.v', units='kn')[-1, 0]

tf_rto = sim_case.get_val('traj.rto.timeseries.time', units='s')[-1, 0]
rf_rto = sim_case.get_val('traj.rto.timeseries.r', units='ft')[-1, 0]

axes[0].annotate(f'field length = {r[-1, 0]:5.1f} ft', xy=(t[-1, 0], r[-1, 0]),
                 xycoords='data', xytext=(0.7, 0.5),
                 textcoords='axes fraction', arrowprops=dict(arrowstyle='->'),
                 horizontalalignment='center', verticalalignment='top')

axes[0].annotate('', xy=(tf_rto, rf_rto),
                 xycoords='data', xytext=(0.7, 0.5),
                 textcoords='axes fraction', arrowprops=dict(arrowstyle='->'),
                 horizontalalignment='center', verticalalignment='top')

axes[1].annotate(f'$v1$ = {v1:5.1f} kts', xy=(tv1, v1), xycoords='data', xytext=(0.5, 0.5),
                 textcoords='axes fraction', arrowprops=dict(arrowstyle='->'),
                 horizontalalignment='center', verticalalignment='top')

plt.legend()
plt.show()

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('traj.rto.states:r')[-1],
                  p.get_val('traj.climb_to_obstacle_clearance.states:r')[-1],
                  tolerance=0.01);

## References

```{bibliography}
:filter: docname in docnames
```